In [2]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [3]:
# 1. 데이터 불러오기
df = pd.read_csv("../../data/DRM-QUAKEOUTSHELTER/DRM-QUAKEOUTSHELTER.csv", encoding='cp949')
print("원본 크기:", df.shape)
display(df.head())

# 2. 컬럼 정리
df = df.rename(columns={"시군구명": "gu"})

# 3. 수치형 변환 (시설면적)
df["시설면적"] = pd.to_numeric(df["시설면적"], errors="coerce").fillna(0)

# 4. 구별 집계
agg = df.groupby("gu").agg(
    quakeoutshelter_count=("수용시설명", "count"),
    quakeoutshelter_total_area=("시설면적", "sum"),
    quakeoutshelter_avg_area=("시설면적", "mean")
).reset_index()

display(agg.head())

# 5. 점수화 (많을수록 안전)
scaler = MinMaxScaler(feature_range=(0,100))
scaled = scaler.fit_transform(agg[["quakeoutshelter_count",
                                   "quakeoutshelter_total_area",
                                   "quakeoutshelter_avg_area"]])

score_df = agg[["gu"]].copy()
for i, col in enumerate(["quakeoutshelter_count",
                         "quakeoutshelter_total_area",
                         "quakeoutshelter_avg_area"]):
    score_df[col+"_score"] = scaled[:, i]

# 6. 종합 점수 (평균)
score_df["quakeoutshelter_mean_score"] = score_df.filter(like="_score").mean(axis=1)

# 7. 저장
score_df.to_csv("../../result/DRM/DRM-QUAKEOUTSHELTER.csv", index=False, encoding="utf-8-sig")

score_df.head()

원본 크기: (1585, 14)


,시설번호,지역코드,시설일련번호,시도명,시군구명,수용시설명,법정동코드,행정동코드,상세주소,시설면적,경도,위도,X좌표,Y좌표
0,47,1114000000,23,서울특별시,중구,충무초등학교 운동장,1114014400,1114058000,서울특별시 중구 퇴계로50길 13(장충동2가),3863.0,127.001883,37.562261,200166.333261,551414.319071
1,48,1135000000,56,서울특별시,노원구,공연초등학교 운동장,1135010300,1135059500,서울특별시 노원구 동일로192가길 16(공릉동),3800.0,127.074461,37.626950,206573.119646,558596.676782
2,49,1135000000,57,서울특별시,노원구,연촌초등학교 운동장,1135010400,1135061100,서울특별시 노원구 공릉로58길 36(하계동),3680.0,127.072801,37.634747,206425.874982,559461.919531
3,50,1135000000,53,서울특별시,노원구,월계고등학교 운동장,1135010200,1135057000,서울특별시 노원구 초안산로5길 66(월계동),1760.0,127.048725,37.631805,204300.988148,559134.020114
4,51,1114000000,52,서울특별시,중구,덕수초등학교 운동장,1114016700,1114052000,서울특별시 중구 덕수궁길 140(정동),2536.0,126.974526,37.568468,197749.486468,552103.540651


,gu,quakeoutshelter_count,quakeoutshelter_total_area,quakeoutshelter_avg_area
0,강남구,120,506340.0,4219.500000
1,강동구,53,293460.0,5536.981132
2,강북구,51,820490.6,16088.050980
3,강서구,116,507293.0,4373.215517
4,관악구,102,397374.0,3895.823529


,gu,quakeoutshelter_count_score,quakeoutshelter_total_area_score,quakeoutshelter_avg_area_score,quakeoutshelter_mean_score
0,강남구,100.000000,55.790585,10.028874,55.273153
1,강동구,33.663366,25.832659,20.016214,26.504080
2,강북구,31.683168,100.000000,100.000000,77.227723
3,강서구,96.039604,55.924698,11.194135,54.386145
4,관악구,82.178218,40.456146,7.575201,43.403189
